# Fully Degenerate Path Probability

In some system having time-homogeneous transition rate matrix $R$ and initial probability distribution $p(x_{0},t_{0})$, the probability of a path $p(\mathcal{C}_{n})$ can be determined such that it includes the probability of all trajectories that follow that path:
\begin{equation}
p(\mathcal{C}_{n})=p(x_{0},t_{0}) \prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \label{pcn}
\end{equation}
This formula uses the transition rates among states $w(x_{i+1} \vert x_{i})$ as given by the transition rate matrix and the total escape rate from a state:
\begin{equation}
w_{x_{i}} = \sum\limits_{\substack{j=0 \\ \neq i}}\limits^{n} w(x_{j} \vert x_{i})
\end{equation}
This path probability formula has a variety of uses. However, it lacks information about the completion time of the trajectories that follow that path. Conversely, a contracted path probability includes the probability of only those trajectories following that path that complete within some amount of time $\tau$. 

This contracted path probability is a consequence of the wait time distribution at each state. The wait time is the amount of time the system stays in one state before transitioning to the next state along the path. The distribution of wait times from a particular state $x_{i}$ is an exponential distribution with respect to the escape rate from that state:
\begin{equation}
\rho(\Delta t_{i} \vert x_{i}) = w_{x_{i}} e^{-\Delta t_{i} w_{x_{i}}}
\end{equation}
The path probaility is given by the convolution of each of the wait time distributions along the path times the path probability in the first equation:
\begin{align}
p(\mathcal{C}_{n},\tau) &=p(x_{0},t_{0})\prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \rho(\Delta t_{0} \vert x_{0})*\rho(\Delta t_{1} \vert x_{1})*...*\rho(\Delta t_{n-1} \vert x_{n-1})*e^{-w_{x_{n}}(t-t_{n})} \\
&= p(x_{0},t_{0})\prod\limits_{i=0}\limits^{n-1} \frac{w(x_{i+1} \vert x_{i})}{w_{x_{i}}} \int_{t_{0}}^{t} dt_{1} w_{x_{0}} e^{-w_{x_{0}}(t_{1}-t_{0})} \times ... \times \int_{t_{n-1}}^{t} dt_{n} w_{x_{n-1}} e^{-w_{x_{n-1}}(t_{n}-t_{n-1})} \times e^{-w_{x_n}(t-t_{n})}
\end{align}
This contracted path probability has a general form that applies to all paths.  However, this formula can be simplified in two cases:

$\bullet$ fully degenerate path

$\bullet$ non-degenerate path

A path can be fully degenerate, partially degenerate, or non-degenerate based upon the escape rates of the states along the path. A path can be fully degenerate without all of its states being the same, as two states can have the same escape rate. By extension, a path must not include two instances of the same state or of two states with the same escape rate in order for it to be non-degenerate. The fully degenerate path probability is given by
\begin{equation}
p(\mathcal{C}_{n},\tau) = p(x_{0},t_{0})\sum\limits_{i=0}\limits^{n-1} w(x_{i+1} \vert x_{i}) \frac{(t-t_{0})^{n} e^{-w(t-t_{0})}}{n!} .
\end{equation}
This path probability is closely related to the Erlang distribution:
\begin{equation}
\rho (x \in [ 0,\infty )) = \frac{\lambda^{k} x^{k-1} e^{-\lambda x}}{(k-1)!}
\end{equation}
when we consider the support to be $\tau=t-t_{0}=x \in [ 0,\infty )$, the shape parmeter $k$ to be the path length $n$, and the rate parameter $w_{x}=w=\lambda$. Due to this similarity, we know that the mean path occurrence time $\langle \tau \rangle_{\mathcal{C}_{n}}$ is given by the mean of the Erlang distribution
\begin{equation}
\langle \tau \rangle_{\mathcal{C}_{n}} = \frac{n}{w_{x}},
\end{equation}
and the variance in the path occurrence time $\sigma^{2}_{\mathcal{C}_{n}} (\tau)$ is given by the variance of the Erlang distribution:
\begin{equation}
\sigma^{2}_{\mathcal{C}_{n}} = \frac{n}{w_{x}^{2}}.
\end{equation}

# Example

As an example, we consider a 4-state Markov model in which all transition rates are $w_{x}=2$. Thus, all escape rates are the same. Every path evolved for this model is fully degenerate. We shall look at the contracted path probabilities, mean path occurrence times, and variance of path occurrence times for three lengths of paths.

In [3]:
# Modules
import math

# Function to determine fully degenerate path probability
def det_prob(wxn,n,tau,wcon,initialp):
    # Initial probability
    pr = initialp
    # Conditional probabilities
    for i in range(0,n):
        pr*=wcon
    # Factorial
    for i in range(1,n+1):
        pr/=i
    # Power of tau
    pr*=math.pow(tau,n)
    # Exponential
    pr*=math.exp(-1*wxn*tau)
    # Return result
    return pr
    
# Function to determine mean path occurrence time
def det_mean(wxn,n):
    # Determine mean
    mn = n/wxn
    # Return result
    return mn
    
# Function to determine variance of path occurrence time
def det_var(wxn,n):
    # n values
    vr = n
    # Square of escape rate
    vr/=math.pow(wxn,n)
    # Return result
    return vr
    
# Wrapper function
def main():
    # Time scale of interest
    dt = 1
    # System
    pinit = [1,0,0,0]
    R = [[0 for i in range(4)] for j in range(4)]
    R[0][:] = [-6,2,2,2]
    R[1][:] = [2,-6,2,2]
    R[2][:] = [2,2,-6,2]
    R[3][:] = [2,2,2,-6]
    # Determine escape rates
    M = len(pinit)
    wx = [0]*(M)
    for i in range(0,M):
        wx[i] = -1*R[i][i]
    # Arrays for results
    probs = [0]*(3)
    means = [0]*(3)
    vari = [0]*(3)
    # Main determinations
    for n in range(2,5):
        tau = (n-1)
        tau*=dt
        probs[n-2] = det_prob(wx[0],n,tau,R[0][1],pinit[0])
        means[n-2] = det_mean(wx[0],n)
        vari[n-2] = det_var(wx[0],n)
        # Printing
        print("For a path of length n=",n)
        print("Path probability:")
        print(probs[n-2])
        print("Mean occurrence time:")
        print(means[n-2])
        print("Variance of occurrence time:")
        print(vari[n-2])
        
main()

For a path of length n= 2
Path probability:
0.004957504353332717
Mean occurrence time:
0.3333333333333333
Variance of occurrence time:
0.05555555555555555
For a path of length n= 3
Path probability:
6.553826510216756e-05
Mean occurrence time:
0.5
Variance of occurrence time:
0.013888888888888888
For a path of length n= 4
Path probability:
8.22418906214482e-07
Mean occurrence time:
0.6666666666666666
Variance of occurrence time:
0.0030864197530864196
